In [19]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [20]:
import sklearn
sklearn.__version__

'0.20.2'

In [21]:
import pandas as pd
import pickle as pkl

In [22]:
%run utils.ipynb
%run targets.ipynb

In [23]:
file = 'ETHBTC_1m_1519496760000_1549568220000.csv'

In [24]:
# Get Data
df = pd.read_csv('../data/'+file)
df.shape
df.head(10)

(500000, 12)

,OPEN_TIME,OPEN,HIGH,LOW,CLOSE,VOLUME,CLOSE_TIME,QUOTE_VOL,NUM_TRADES,TAKE_BASE_VOL,TAKE_QUOTE_VOL,EXTRA
0,1519496760000,0.085541,0.085570,0.085407,0.085473,27.714,1519496819999,2.370345,192,15.308,1.309733,0
1,1519496820000,0.085473,0.085570,0.085448,0.085449,20.128,1519496879999,1.720609,122,6.579,0.562654,0
2,1519496880000,0.085448,0.085569,0.085400,0.085538,45.510,1519496939999,3.890176,209,19.426,1.660960,0
3,1519496940000,0.085538,0.085546,0.085402,0.085540,22.231,1519496999999,1.900513,105,14.320,1.224483,0
4,1519497000000,0.085539,0.085552,0.085464,0.085464,15.243,1519497059999,1.303412,117,8.373,0.716219,0
5,1519497060000,0.085466,0.085555,0.085464,0.085545,31.895,1519497119999,2.727413,131,19.237,1.645547,0
6,1519497120000,0.085545,0.085545,0.085464,0.085539,13.325,1519497179999,1.139346,78,7.311,0.625352,0
7,1519497180000,0.085539,0.085555,0.085414,0.085517,28.202,1519497239999,2.411093,195,14.409,1.232346,0
8,1519497240000,0.085460,0.085545,0.085419,0.085534,15.302,1519497299999,1.307959,134,7.216,0.617035,0
9,1519497300000,0.085440,0.085545,0.085440,0.085533,13.730,1519497359999,1.173580,105,6.186,0.529001,0


In [25]:
data = df.loc[:, ['OPEN_TIME', 'CLOSE_TIME', 'OPEN', 'HIGH', 'LOW', 'CLOSE']]
data.shape
data[:5]

(500000, 6)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464


In [26]:
# Get Target
%run targets.ipynb
target = get_target_1('../data/'+file)
target.shape
target.head()

(499999, 2)

,CLOSE_TIME,TARGET
0,1519496819999,1.0
1,1519496879999,0.0
2,1519496939999,0.0
3,1519496999999,1.0
4,1519497059999,0.0


In [27]:
data = pd.merge(data, target, on='CLOSE_TIME')
data.shape
data.head()

(499999, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473,1.0
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449,0.0
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538,0.0
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540,1.0
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464,0.0


In [28]:
TR_split, CV_split, TT_split = TR_CV_TT_split(data, 0.8, 0, 0.2)

TR_split.shape
CV_split.shape
TT_split.shape

TR_split.head()

(399999, 7)

(0, 7)

(100000, 7)

,OPEN_TIME,CLOSE_TIME,OPEN,HIGH,LOW,CLOSE,TARGET
0,1519496760000,1519496819999,0.085541,0.085570,0.085407,0.085473,1.0
1,1519496820000,1519496879999,0.085473,0.085570,0.085448,0.085449,0.0
2,1519496880000,1519496939999,0.085448,0.085569,0.085400,0.085538,0.0
3,1519496940000,1519496999999,0.085538,0.085546,0.085402,0.085540,1.0
4,1519497000000,1519497059999,0.085539,0.085552,0.085464,0.085464,0.0


In [29]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler(copy=True, with_mean=True, with_std=True)
window = 30
def data_transformer(id, data, target, retrain=False):
    global scaler
    global window
    
    if retrain==True:
        scaler = scaler.fit(data)
        
    data = scaler.transform(data)
    return get_multiple_ts_dataset(id, data, target, tr_win=window, tt_win=0, point_target=True)

In [30]:
TR_id     = TR_split.iloc[:, :2  ].values
TR_data   = TR_split.iloc[:, 2:-1].values
TR_target = TR_split.iloc[:, -1: ].values

TT_id     = TT_split.iloc[:, :2  ].values
TT_data   = TT_split.iloc[:, 2:-1].values
TT_target = TT_split.iloc[:, -1: ].values

TR_id.shape
TR_data.shape
TR_target.shape

TT_id.shape
TT_data.shape
TT_target.shape

(399999, 2)

(399999, 4)

(399999, 1)

(100000, 2)

(100000, 4)

(100000, 1)

In [32]:
%run hyperopt.ipynb
%run models.ipynb

In [33]:
hyperopt = Hyperopt(BayesR())
best_loss, best_params = hyperopt.run(
    TR_id, TR_data, TR_target, 
    eval_set={
        'TT': (TT_id, TT_data, TT_target)
    },
    n_cv=3,
    algo='grid',
    data_transformer=data_transformer,
    max_evals=1
)

Hyperopt Iteration : 1
{}
Convergence after  10  iterations
('CV_fold : ', 0, 7.571086883544922)
         acc       auc        f1   log_loss
TT  0.516110  0.500924  0.057954  16.712992
TR  0.526870  0.510024  0.190115  16.341370
CV  0.518667  0.504315  0.118165  16.624694
Convergence after  10  iterations
('CV_fold : ', 1, 8.811187982559204)
         acc       auc        f1   log_loss
TT  0.516540  0.501871  0.087180  16.698142
TR  0.525327  0.509031  0.183045  16.394661
CV  0.517053  0.500223  0.090385  16.680414
Convergence after  11  iterations
('CV_fold : ', 2, 10.295515060424805)
         acc       auc        f1   log_loss
TT  0.516230  0.501324  0.073807  16.708849
TR  0.523783  0.507002  0.155623  16.447989
CV  0.527109  0.501349  0.055670  16.333100
----- Metrics Mean -----
         acc       auc        f1   log_loss
TT  0.516293  0.501373  0.072980  16.706661
TR  0.525327  0.508686  0.176261  16.394674
CV  0.520943  0.501962  0.088073  16.546069
----- Metrics Std -----
       

In [15]:
from sklearn.linear_model import BayesianRidge

TR_id, TR_X, TR_y = data_transformer(TR_id, TR_data, TR_target, retrain=True)
TT_id, TT_X, TT_y = data_transformer(TT_id, TT_data, TT_target, retrain=False)

TR_X.shape
TT_X.shape

model = BayesianRidge(**best_params)
model = model.fit(TR_X, TR_y)

TR_y_pred = model.predict(TR_X)
TT_y_pred = model.predict(TT_X)

TR_metrics = eval_class_metrics(TR_y, TR_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])
TT_metrics = eval_class_metrics(TT_y, TT_y_pred, metrics=['auc', 'f1', 'acc', 'log_loss'])

res = pd.DataFrame({x:[] for x in ['auc', 'f1', 'acc', 'log_loss']})
res = res.append(pd.Series(TR_metrics, name='TR'))
res = res.append(pd.Series(TT_metrics, name='TT'))

print(res)

(399700, 1200)

(99701, 1200)

Convergence after  41  iterations
         acc       auc        f1   log_loss
TR  0.524433  0.505954  0.122330  16.425515
TT  0.515943  0.500725  0.049175  16.718758


In [16]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [17]:
nb_name = 'Simple_BayesR_Grid'

In [18]:
data_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'intv':'1m', 
    'cols':'OHCL',
    'win':window
}.iteritems()])

proc_tag = '__'.join([str(x)+'_'+str(y) for x, y in {
    'model': nb_name.replace('.ipynb', ''),
    'TT_accuracy':'{:.4f}'.format(TT_metrics['acc']),
    'CV_accuracy':'{:.4f}'.format(best_loss)
}.iteritems()])

import time
time_tag = time.strftime('%y%m%d_%H%M%S')

big_tag = time_tag+'_|_'+data_tag+'_|_'+proc_tag

data_dump = {
    'best_loss': best_loss,
    'best_params': best_params,
    'model': model,
    'TR_id': TR_id,
    'TR_y_pred': TR_y_pred,
    'TT_id': TT_id,
    'TT_y_pred': TT_y_pred,
    'results': res
}

f = open('../models/'+time_tag+'.pkl', 'wb')
pkl.dump(data_dump, f, -1)
f.close()

%notebook -e ../models/{big_tag}.ipynb